In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_3258965/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp = df_tmp[:1000]
df_tmp 

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
4,"MARIA VELIZ ALCAYAGA, VALLENAR, ATACAMA, CHILE...",VALLENAR,None,,
8,UNO 11 POSTE 669700 MINA CARACOLES RINCONADA,SAN ESTEBAN,None,,
11,PJE. QUILLAY 1485 POBLACION EL BOSQUE DE OCALI...,ANGOL,None,,
13,LOS CLAVELES 66.ERCILLA.LA ARAUCANIA. VILLA MI...,ERCILLA,None,,
16,EMA QUIROGA LAGUNA VERDE,VALPARAISO,None,,
...,...,...,...,...,...
5005,ALFALFARES PARCELA 33 SITIO N1SUR,LA SERENA,None,,
5018,LOS FAISANES SITIO 23 LA CRUZ LO ROJAS AL LADO...,LA CRUZ,None,,
5024,PASAJE COLCUMA SN.TIRUA.BIOBIO. TIRUA,TIRUA,None,,
5039,LA HABANA 6876 ENTRE CARDENAL JOSE MARIA CARO ...,LO ESPEJO,None,,


In [6]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

# Assuming df is your original Pandas DataFrame
ddata = dd.from_pandas(df_tmp, npartitions=30)

# Sample meta dataframe to indicate the structure of the output
meta = pd.DataFrame({
    "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
    "COMUNA DOMICILIO": pd.Series(dtype='str'),
    "NUEVA DIRECCION": pd.Series(dtype='str'),
    "X": pd.Series(dtype='float'),
    "Y": pd.Series(dtype='float')
})

def apply_validate_address(df_chunk):
    # Handle missing values before passing to validate_address
    df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
    
    # Apply the validation function to each row and unpack the results
    df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
        lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
        axis=1
    )
    return df_chunk

# Map the function to each partition, with the meta keyword to define the structure
res = ddata.map_partitions(apply_validate_address, meta=meta).compute()

sin coincidencias ['EL MANZANO', 'SAN BERNARDO']  ------   EL MANZANO SAN BERNARDO
['SAN ANDRES', '2133', 'CURICO', 'MAULE', 'POBLACION', 'BOMBERO', 'GARRIDO']  ------  SAN ANDRES 2133.CURICO.MAULE. POBLACION BOMBERO GARRIDO CURICO  -------  Avenida Colón, Población Bombero Garrido, Curicó, Provincia de Curicó, Región del Maule, 3341353, Chile
['AVDA', 'HONDURAS', '312', 'INEZ', 'SUAREZ', 'VALDIVIA']  ------  AVDA HONDURAS 312 INEZ DE SUAREZ VALDIVIA  -------  Templo de Santiago, Chile, Avenida Pocuro, Barrio Los Estanques, Providencia, Provincia de Santiago, Región Metropolitana de Santiago, 7500000, Chile
['ARTURO', 'PRAT', '617', 'D', 'A', 'STA', 'BARBARA', 'SANTA']  ------  ARTURO PRAT 617 D.A STA BARBARA STA BARBARA SANTA BARBARA  -------  Arturo Prat, Villa Santa Elsa, Playas Blancas, Las Cruces, El Tabo, Provincia de San Antonio, Región de Valparaíso, 2690000, Chile
['SAN MARCOS', '2615', 'CON', 'SANTA', 'DANIELA', 'PUENTE', 'ALTO']  ------  SAN MARCOS 2615 SAN MARCOS CON SANTA 

In [7]:
res

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
4,"MARIA VELIZ ALCAYAGA, VALLENAR, ATACAMA, CHILE...",VALLENAR,"María Véliz Alcayaga, Villa de Ballenary, Vall...",-28.587871,-70.746431
8,UNO 11 POSTE 669700 MINA CARACOLES RINCONADA,SAN ESTEBAN,"Pasaje Uno, Mina Caracoles, Rinconada, Provinc...",-32.834792,-70.712817
11,PJE. QUILLAY 1485 POBLACION EL BOSQUE DE OCALI...,ANGOL,"Sabor con Amor, 582, Los Carolinos, Villa Los ...",-33.572507,-70.679039
13,LOS CLAVELES 66.ERCILLA.LA ARAUCANIA. VILLA MI...,ERCILLA,"Los Claveles, Villa Los Robles, San Pedro de l...",-36.831533,-73.114782
16,EMA QUIROGA LAGUNA VERDE,VALPARAISO,None,NaN,NaN
...,...,...,...,...,...
5005,ALFALFARES PARCELA 33 SITIO N1SUR,LA SERENA,"Parcela 34, Villa El Romero II, Colonia Alfalf...",-29.897488,-71.220879
5018,LOS FAISANES SITIO 23 LA CRUZ LO ROJAS AL LADO...,LA CRUZ,"Los Faisanes, Lo Rojas, La Cruz, Provincia de ...",-32.813535,-71.248354
5024,PASAJE COLCUMA SN.TIRUA.BIOBIO. TIRUA,TIRUA,None,NaN,NaN
5039,LA HABANA 6876 ENTRE CARDENAL JOSE MARIA CARO ...,LO ESPEJO,"6876, La Habana, José María Caro Rodríguez, Lo...",-33.508902,-70.692577


In [8]:
print("A")

A


In [9]:
res.to_csv('./data/out/ad1.csv')